In [28]:
from datetime import datetime

from db_comm import PortfolioDBManager, DATABASE_NAME

In [29]:
pm = PortfolioDBManager(DATABASE_NAME)

Calculation Motor

In [30]:
from portfolio_test.motor import CalculationMotor

In [33]:
cm = CalculationMotor("AAPL")

In [35]:
# withdraw cash
pm.record_transaction(
    tx_type="WITHDRAW", # WITHDRAW or DEPOSIT
    ticker="CASH",
    shares=pm.get_cash_balance(),
    actual_price=1,
    tx_datetime=datetime.now(),
)
pm.record_transaction(
    tx_type="DEPOSIT", # WITHDRAW or DEPOSIT
    ticker="CASH",
    shares=100,
    actual_price=1,
    tx_datetime=datetime.now(),
)

2025-12-18 21:53:33: ✅ Recorded WITHDRAW: CASH @ 101.6188. Snapshot updated.
2025-12-18 21:53:33: ✅ Recorded DEPOSIT: CASH @ 100.0. Snapshot updated.


Build calculation motor for AAPL

In [36]:
trade = cm.df
trade

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
Date,,,,,,,,
2020-01-02,688.757977,698.895014,686.316756,698.313764,72.468269,135480400,0.0,0.0
2020-01-03,690.873736,698.848469,689.362500,691.524733,71.763718,146322800,0.0,0.0
2020-01-06,683.061770,697.406980,680.643750,697.034972,72.335556,118387200,0.0,0.0
2020-01-07,697.127991,699.592486,691.641026,693.756784,71.995354,108872000,0.0,0.0
2020-01-08,690.897009,707.823006,690.897009,704.916756,73.153473,132079200,0.0,0.0
...,...,...,...,...,...,...,...,...
2025-12-11,2595.630057,2600.186966,2546.432977,2585.678989,278.029999,33248000,0.0,0.0
2025-12-12,2584.469943,2596.746011,2574.426068,2588.003989,278.279999,39532900,0.0,0.0
2025-12-15,2605.394943,2605.394943,2537.411966,2549.222864,274.109985,50409100,0.0,0.0


In [37]:
import numpy as np

for date in trade.index:
    print(f"- - - - - Date: {date.date()} - - - - -")
    # if price today is lower than yesterday, buy as much as possible
    trade['Open_ydy'] = trade['Open'].shift(1)
    if date != trade.index.min():
        price = trade.Open.loc[date]  # price in SEK
        price_yesterday = trade.Open_ydy.loc[date]

        if price < price_yesterday:
            #print("Price dropped, buying AAPL")
            cash_balance = pm.get_cash_balance()
            shares_to_buy = np.divide(cash_balance, price)
            if shares_to_buy > 0:
                print("Buying AAPL")
                pm.record_transaction(
                    tx_type="BUY",
                    ticker="AAPL",
                    shares=shares_to_buy,
                    actual_price=price,
                    tx_datetime=date,
                    currency="SEK",
                )
        # if price today is higher than yesterday, sell all shares
        elif price > price_yesterday:
            portfolio = pm.get_portfolio_snapshot()
            if "AAPL" in list(portfolio["ticker"]):
                shares_to_sell = portfolio[portfolio["ticker"] == "AAPL"]["net_shares"].iloc[0]
                if shares_to_sell > 0:
                    pm.record_transaction(
                        tx_type="SELL",
                        ticker="AAPL",
                        shares=shares_to_sell,
                        actual_price=price,
                        tx_datetime=date,
                        currency="SEK",
                    )


- - - - - Date: 2020-01-02 - - - - -
- - - - - Date: 2020-01-03 - - - - -
- - - - - Date: 2020-01-06 - - - - -
Buying AAPL
   -> Cash balance adjusted by -100.0
2020-01-06 00:00:00: ✅ Recorded BUY: AAPL @ 100.0. Snapshot updated.
- - - - - Date: 2020-01-07 - - - - -
   -> Cash balance adjusted by 102.0593
2020-01-07 00:00:00: ✅ Recorded SELL: AAPL @ 102.0593. Snapshot updated.
- - - - - Date: 2020-01-08 - - - - -
Buying AAPL
   -> Cash balance adjusted by -102.0593
2020-01-08 00:00:00: ✅ Recorded BUY: AAPL @ 102.0593. Snapshot updated.
- - - - - Date: 2020-01-09 - - - - -
   -> Cash balance adjusted by 105.5213
2020-01-09 00:00:00: ✅ Recorded SELL: AAPL @ 105.5213. Snapshot updated.
- - - - - Date: 2020-01-10 - - - - -
- - - - - Date: 2020-01-13 - - - - -
- - - - - Date: 2020-01-14 - - - - -
- - - - - Date: 2020-01-15 - - - - -
Buying AAPL
   -> Cash balance adjusted by -105.5213
2020-01-15 00:00:00: ✅ Recorded BUY: AAPL @ 105.5213. Snapshot updated.
- - - - - Date: 2020-01-16 - - - - 

In [38]:
pm.get_portfolio_snapshot()

,ticker,net_shares,last_trade_price,total_position_value
0,CASH,193.0534,1.0,193.0534


In [42]:
# find Price_SEK at max date
last_price = trade.loc[trade.index.max(), "Open"]
first_price = trade.loc[trade.index.min(), "Open"]
earnings = pm.get_portfolio_snapshot()["total_position_value"].sum()
holding_gain = last_price/first_price
trading_gain = earnings/100
print(trading_gain)
print(holding_gain)
print(f"Last price: {(holding_gain-1)*100:.2f} %, Earnings: {(trading_gain-1)*100:.2f} %")
if trading_gain < holding_gain:
    print("Overall worse than buy and hold. "
          f"Specifically, just buying would have given you {((holding_gain-1)/(trading_gain-1)):.2f} times better results.")
else:
    print(f"Overall better than buy and hold")
    if holding_gain-1 > 0:
        print(f"{((trading_gain-1)/(holding_gain-1)):.2f} times better than buy and hold")
    elif trading_gain-1 > 0:
        print("Holding was a loss, but trading made a gain")
    else:
        print("Both trading and holding were a loss but trading lost less")

1.9305340000000002
3.7133407889737833
Last price: 271.33 %, Earnings: 93.05 %
Overall worse than buy and hold. Specifically, just buying would have given you 2.92 times better results.
